In [17]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [18]:
import math
import numpy as np

import cv2
from os import listdir
from scipy import ndimage, misc
import os, glob
import sys
import skimage
from skimage import feature, data, io

from scipy.misc import imread, imsave
from PIL import Image
import random

In [39]:
def get_file_name_dir(rootDir, ext):
 
    # Return file names with certain extension

    return glob.glob1(rootDir, "*." + ext)


def clip(img, dtype, maxval):
    return np.clip(img, 0, maxval).astype(dtype)

def HorizontalFlip(img):
    img = cv2.flip(img, 1)
    return img


def VerticalFlip(img):
    img = cv2.flip(img, 0)
    return img

def RandomRotate(img):
    factor = random.randint(0, 4)
    img = np.rot90(img, factor)
    return img

def RandomContrast(img):
    limit=.1

    alpha = 1.0 + limit * random.uniform(-1, 1)

    gray = cv2.cvtColor(img[:, :, :3], cv2.COLOR_BGR2GRAY)
    gray = (3.0 * (1.0 - alpha) / gray.size) * np.sum(gray)
    maxval = np.max(img[..., :3])
    dtype = img.dtype
    img[:, :, :3] = clip(alpha * img[:, :, :3] + gray, dtype, maxval)
    return img

def RandomBrightness(img):

    limit=.1

    alpha = 1.0 + limit * random.uniform(-1, 1)

    maxval = np.max(img[..., :3])
    dtype = img.dtype
    img[..., :3] = clip(alpha * img[..., :3], dtype, maxval)
    return img

def CenterCrop(img):
    limit=.1
    height, width = (450,450)
    h, w, c = img.shape
    dy = (h - height) // 2
    dx = (w - width) // 2

    y1 = dy
    y2 = y1 + height
    x1 = dx
    x2 = x1 + width
    img = img[y1:y2, x1:x2]

    return img

def RandomHueSaturationValue(img):

    hue_shift_limit=(-10, 10)
    sat_shift_limit=(-25, 25)
    val_shift_limit=(-25, 25)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img)
    hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
    h = cv2.add(h, hue_shift)
    sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
    s = cv2.add(s, sat_shift)
    val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
    v = cv2.add(v, val_shift)
    img = cv2.merge((h, s, v))
    img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    return img

def RandomSaturation(img):
#     def __init__(self, limit=0.3, prob=0.5):
#         self.limit = limit
#         self.prob = prob

#     def __call__(self, img):
#         # dont work :(
#         if random.random() < self.prob:
    limit=0.3
    maxval = np.max(img[..., :3])
    dtype = img.dtype
    alpha = 2.0 + random.uniform(-limit, limit)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    img[..., :3] = alpha * img[..., :3] + (1.0 - alpha) * gray
    img[..., :3] = clip(img[..., :3], dtype, maxval)
    return img

def RandomFilter(img):
    """
    blur sharpen, etc
    """

    limit=.5
    alpha = limit * random.uniform(0, 1)
    kernel = np.ones((3, 3), np.float32) / 9 * 0.2

    colored = img[..., :3]
    colored = alpha * cv2.filter2D(colored, -1, kernel) + (1 - alpha) * colored
    maxval = np.max(img[..., :3])
    dtype = img.dtype
    img[..., :3] = clip(colored, dtype, maxval)

    return img

def ShiftScaleRotate(img):
    shift_limit=0.0625
    scale_limit=0.1
    rotate_limit=45

    height, width, channel = img.shape

    angle = random.uniform(-rotate_limit, rotate_limit)
    scale = random.uniform(1 - scale_limit, 1 + scale_limit)
    dx = round(random.uniform(-shift_limit, shift_limit)) * width
    dy = round(random.uniform(-shift_limit, shift_limit)) * height

    cc = math.cos(angle / 180 * math.pi) * scale
    ss = math.sin(angle / 180 * math.pi) * scale
    rotate_matrix = np.array([[cc, -ss], [ss, cc]])

    box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
    box1 = box0 - np.array([width / 2, height / 2])
    box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

    box0 = box0.astype(np.float32)
    box1 = box1.astype(np.float32)
    mat = cv2.getPerspectiveTransform(box0, box1)
    img = cv2.warpPerspective(img, mat, (width, height),
                              flags=cv2.INTER_LINEAR,
                              borderMode=cv2.BORDER_REFLECT_101)
    return img
def ShiftScale(img):
    limit=4

    height, width, channel = img.shape
    assert (width == height)
    size0 = width
    size1 = width + 2 * limit
    size = round(random.uniform(size0, size1))

    dx = round(random.uniform(0, size1 - size))
    dy = round(random.uniform(0, size1 - size))

    y1 = dy
    y2 = y1 + size
    x1 = dx
    x2 = x1 + size

    img1 = cv2.copyMakeBorder(img, limit, limit, limit, limit, borderType=cv2.BORDER_REFLECT_101)
    img = (img1[y1:y2, x1:x2, :] if size == size0
    else cv2.resize(img1[y1:y2, x1:x2, :], (size0, size0), interpolation=cv2.INTER_LINEAR))

    return img

def shift(img):
    limit=4
    dx = round(random.uniform(-limit, limit))
    dy = round(random.uniform(-limit, limit))

    height, width, channel = img.shape
    y1 = limit + 1 + dy
    y2 = y1 + height
    x1 = limit + 1 + dx
    x2 = x1 + width

    img1 = cv2.copyMakeBorder(img, limit + 1, limit + 1, limit + 1, limit + 1,
                              borderType=cv2.BORDER_REFLECT_101)
    img = img1[y1:y2, x1:x2, :]


    return img

def Rotate(img):
    limit=10 #10,45, 90,270

    angle = random.uniform(-limit, limit)

    height, width = img.shape[0:2]
    mat = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1.0)
    img = cv2.warpAffine(img, mat, (height, width),
                         flags=cv2.INTER_LINEAR,
                         borderMode=cv2.BORDER_REFLECT_101)

    return img

def Transpose(img):
    img = img.transpose(1, 0, 2)
    return img

In [40]:
input_path = 'data/data_practice/AK/'
input_extension = 'jpg'
inputs_files = sorted(get_file_name_dir(input_path, input_extension))

cont = 1
for file_name in inputs_files:
    print('Preprocessing: ' + file_name + ': ' + str(cont))
    cont += 1
    img = input_path + file_name

#     if file_name.startswith('VF_'):
    im_in = cv2.imread(img)
    image = Transpose(im_in)
    cv2.imwrite(input_path + 'Transpose_' +file_name , image)

Preprocessing: ISIC_0063250.jpg: 1
Preprocessing: ISIC_0063320.jpg: 2
Preprocessing: ISIC_0063536.jpg: 3
Preprocessing: ISIC_0063883.jpg: 4
Preprocessing: ISIC_0063890.jpg: 5
Preprocessing: ISIC_0064063.jpg: 6
Preprocessing: ISIC_0064330.jpg: 7
Preprocessing: ISIC_0064789.jpg: 8
Preprocessing: ISIC_0065992.jpg: 9
Preprocessing: ISIC_0066006.jpg: 10
Preprocessing: ISIC_0066056.jpg: 11
Preprocessing: ISIC_0066123.jpg: 12
Preprocessing: ISIC_0066278.jpg: 13
Preprocessing: ISIC_0066296.jpg: 14
Preprocessing: ISIC_0066306.jpg: 15
Preprocessing: ISIC_0066315.jpg: 16
Preprocessing: ISIC_0067100.jpg: 17
Preprocessing: ISIC_0067204.jpg: 18
Preprocessing: ISIC_0067297.jpg: 19
Preprocessing: ISIC_0067425.jpg: 20
Preprocessing: ISIC_0067500.jpg: 21
Preprocessing: ISIC_0067728.jpg: 22
Preprocessing: ISIC_0067755.jpg: 23
Preprocessing: ISIC_0067783.jpg: 24
Preprocessing: ISIC_0068885.jpg: 25
Preprocessing: ISIC_0068973.jpg: 26
Preprocessing: ISIC_0068977.jpg: 27
Preprocessing: ISIC_0069289.jpg: 28
P